Title of Final Project: Child Mind Institute - Detect Sleep States

Section: 52745


Student Name: Thomas Wynn

Student UT EID: ttw483

Date: 10/30/23


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import polars as pl
import plotly.express as px

import seaborn as sns
from IPython.display import Markdown
import gc

train_events_path = "/Users/thomaswynn/Desktop/sleep_kaggle/child-mind-institute-detect-sleep-states/train_events.csv"
train_series_path = "/Users/thomaswynn/Desktop/sleep_kaggle/child-mind-institute-detect-sleep-states/train_series.parquet"



In [ ]:
print(pd.__version__)

In [ ]:
# Memory reduction for train_series
#The following function has been taken from the notebook
# https://www.kaggle.com/code/renatoreggiani/zzzs-feat-eng-ideas-60-memory-reduction?scriptVersionId=143987308

from pandas.api.types import is_datetime64_ns_dtype
import gc

# import warnings
# warnings.filterwarnings("ignore")

def reduce_mem_usage(df):
    
    """ 
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage of dataframe is {start_mem:.2f} MB')
    
    for col in df.columns:
        if col == 'timestamp' or col == 'date' or col == 'time' : continue
        col_type = df[col].dtype
        if col_type != object and not is_datetime64_ns_dtype(df[col]):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        

    df['series_id'] = df['series_id'].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage after optimization is: {end_mem:.2f} MB')
    decrease = 100 * (start_mem - end_mem) / start_mem
    print(f'Decreased by {decrease:.2f}%')
    
    return df

In [ ]:
## create a list 'dt_transforms' with transformations 
## to be applied to a timestamp column in the data
dt_transforms = [
   pl.col('timestamp').str.to_datetime(), 
   (pl.col('timestamp').str.to_datetime().dt.date()).alias('date'), 
   pl.col('timestamp').str.to_datetime().dt.time().alias('time')
]

trn_series = (
   pl.scan_parquet(train_series_path)
   .with_columns((dt_transforms))
   .collect()
   .to_pandas()
)

trn_events = (
   pl.scan_csv(train_events_path)
   .with_columns((dt_transforms))
   .collect()
   .to_pandas()
)

# # download test_series data
# tst_series = (
#    pl.scan_parquet('/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet')
#    .with_columns((dt_transforms))
#    .collect()
#    .to_pandas()
# )

reduce_mem_usage(trn_series)

In [ ]:
for col in trn_series.columns :
    print(col)

In [ ]:
print(is_datetime64_ns_dtype((trn_series.time[0])))

In [ ]:
no_nan_series = trn_events.groupby('series_id')['step'].apply(lambda x: x.isnull().any())
no_nan_series.value_counts()

In [ ]:
no_nan_series = no_nan_series.drop('31011ade7c0a') # incomplete events data
no_nan_series = no_nan_series.drop('a596ad0b82aa') # incomplete events data

let remove these.

In [ ]:
no_nan_series = no_nan_series[~no_nan_series].index.tolist()
no_nan_series

## In this function, we are combining train_series with train_events to create a binary dataset, for a given series ID. Time points where the individual are asleep are labeled 1 and awake, 0.

In [ ]:
def get_train_series(series): # takes in a series ID and returns
    train_series = pd.read_parquet(train_series_path, filters=[('series_id','==',series)])
    train_events = pd.read_csv(train_events_path).query('series_id == @series')
    
    train_events = train_events.dropna()
    train_events["step"]  = train_events["step"].astype("int")
    train_events["awake"] = train_events["event"].replace({"onset":1,"wakeup":0})

    train = pd.merge(train_series, train_events[['step','awake']], on='step', how='left')
    train["awake"] = train["awake"].bfill(axis ='rows')

    train['awake'] = train['awake'].fillna(1) # awake
    train["awake"] = train["awake"].astype("int")
    return(train)

### Lets visualize this...

In [ ]:
smaller_train_data = []

for series_id in no_nan_series:
    train = get_train_series(series_id)
    smaller_train_data.append(train)

In [ ]:
series_size_dict = {id : size for (id,size) in zip([dataframe.series_id.unique()[0] for dataframe in smaller_train_data], [len(dataframe) for dataframe in smaller_train_data])}

In [ ]:
index = 0
for series_id in no_nan_series : 
    if series_id == "349c5562ee2c":
        print(index)
        break
    else : index += 1
min(series_size_dict, key = series_size_dict.get)
    

In [ ]:
min(series_size_dict, key = series_size_dict.get)

In [ ]:
smaller_train_data[2]

In [ ]:
from matplotlib.widgets import Slider
%matplotlib inline
data = smaller_train_data[-8]
series_id = data.series_id.unique()[0]
series_events = trn_events[trn_events.series_id == series_id]
wake_steps = series_events[series_events.event == 'wakeup'].timestamp
onset_steps = series_events[series_events.event == 'onset'].timestamp

awake_changes = data.awake.diff()


In [ ]:
data.index = pd.to_datetime(data['timestamp'])

enmo_df = data.enmo.resample("60S").mean().to_frame(name = 'enmo')
anglez_df = data.anglez.resample("60S").mean().to_frame(name = 'anglez')

awake_enmo = data.awake.resample("60S").mean()
awake_anglez = data.awake.resample("60S").mean()

awake_enmo = awake_enmo.reset_index()
awake_anglez = awake_anglez.reset_index()

awake_enmo = awake_enmo.drop('timestamp', axis = 1)
awake_anglez = awake_anglez.drop('timestamp', axis = 1)

enmo_df = pd.concat((enmo_df.reset_index(), awake_enmo), axis = 1)
anglez_df = pd.concat((anglez_df.reset_index(), awake_anglez), axis = 1)


def makezero(value) : 
    if value < 1:
        return 0
    else : return 1
        
enmo_df.awake = enmo_df.awake.apply(lambda val : makezero(val))
anglez_df.awake = anglez_df.awake.apply(lambda val : makezero(val))
enmo_df

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()
wake_steps_df = wake_steps.to_frame(name='timestamp')
onset_steps_df = onset_steps.to_frame(name='timestamp')


interval = alt.selection_interval(encodings=['x'])

base = alt.Chart(enmo_df.reset_index()).mark_line().encode(
    x = 'timestamp:T',
    y = 'enmo:Q',
    color = alt.Color('awake:N')
).properties (
    width = 1500,
    height = 250
)

base2 = alt.Chart(anglez_df.reset_index()).mark_line().encode(
    x = 'timestamp:T',
    y = 'anglez:Q',
    color = alt.Color('awake:N')
).properties (
    width = 1500,
    height = 250
)

rule1 = alt.Chart(wake_steps_df).mark_rule(
    color="green",
    strokeWidth=3,
    strokeDash=[20, 1]
).encode(
    x="timestamp:T",
)

rule2 = alt.Chart(onset_steps_df).mark_rule(
    color="red",
    strokeWidth=3,
    strokeDash=[20, 1]
).encode(
    x="timestamp:T",
)


top = base + rule1 + rule2
top2 = base2 + rule1 + rule2

chart = top.encode(
    x=alt.X('timestamp:T', scale=alt.Scale(domain=interval))
)

chart2 = top2.encode(
    x=alt.X('timestamp:T', scale=alt.Scale(domain=interval))
)

view = base.add_selection(
    interval
).properties (
    height = 50,
    width = 1500
)

(chart2 & chart & view).configure_axis(
    grid=False
)


----------------

In [ ]:
import matplotlib.dates as mdates
my_colors = ["#f79256", "#fbd1a2", "#7dcfb6", "#00b2ca"]
trn_events["time"] = trn_events.timestamp.dt.time


df = trn_events.copy()
df.time = pd.to_datetime(df.time, format='%H:%M:%S')
df.set_index(['time'],inplace=True)
df.event = df.event.astype(str)

df.event = df.event.astype(str)

def change_event_numeric(event):
    if event == "onset":
        return 1
    if event == "wakeup":
        return 2
    
df.event = df.event.apply(change_event_numeric)

def jitter(values,j= 0.01):
    return values + np.random.normal(j,0.05,values.shape)


# Create a scatter plot
fig = px.scatter(
    x=df.index,
    y=jitter(df["event"], 0.01),
    color=df["event"],
    size=df["event"] + 4,
    color_continuous_scale = px.colors.qualitative.D3
)

# Set the title and axis labels
fig.update_layout(
    title="Times for onset & wakeup",
    xaxis_title="time",
    yaxis_title="event type",
    width = 1500,
    height = 500,
    yaxis = dict(
        tickmode = 'array',
        tickvals = [1, 2],
        ticktext = ['Onset of Asleep', 'Wake up']
    )
)

fig.update(layout_coloraxis_showscale=False)
fig.update_xaxes(showline=False, linewidth=0, linecolor='black', gridcolor='black', gridwidth= 10)
fig.update_yaxes(showline=False, linewidth=0, linecolor='black', gridcolor='black', gridwidth= 4)

# Rotate the x-axis ticks
fig.update_xaxes(tickangle=45)
fig.show()


-----------------

In [ ]:
def fft_signal_clean(signal, filter_percent=99.7, f=0.2, show_plot=False, y_min=0, y_max=1, L_min_idx=0, L_max_idx=-1):
    """
    Denoising signal using Fast Fourier Transformation
    Adapted from: https://www.youtube.com/watch?v=s2K1JfNR7Sc
    
    All errors are mine.
    """
    t = np.arange(0,1,f)
    
    # Get signal length
    n = len(signal)
    
    # Compute the FFT
    fhat = np.fft.fft(signal, n)
    
    # Compute Power Spectrum
    PSD = fhat * np.conj(fhat) / n
    print(PSD)
    # Create x-axis of frequencies
    freq = (1 / (f * n)) * np.arange(n)
    
    PSD_filter = np.percentile(PSD.real, filter_percent)
    
    # For graphing
    if show_plot:
        L = np.arange(1, np.floor(n/2), dtype='int')
        fig,axs = plt.subplots(1,1)
        plt.sca(axs)
        plt.plot(freq[L], PSD[L], color='c', linewidth=2, label='Noisy')
        axs.axhline(
            y=PSD_filter,
            color='red'
        )
        ## Limits on y-axis
        plt.ylim(y_min, y_max)
        
        ## Limits on x-axis
        plt.xlim(freq[L[L_min_idx]]-0.1, freq[L[L_max_idx]]+0.1)
        plt.legend()
        plt.title("PSD")
        plt.show()
    
    indices = PSD >= PSD_filter
    
    # For graphing
    if show_plot:
        PSD_clean = PSD * indices
    
    fhat = fhat * indices
    
    # Compute inverse FFT
    signal_clean = np.fft.ifft(fhat)
    
    return signal_clean.real

In [ ]:
def plot_series(series, s_lbls, column='enmo'):
    """
    Plot the selected column and the accompanying events
    """
    # assert column in s.columns, "Invalid column."
    
    ymin = series[column].min()
    ymax = series[column].max()
    
    color = '#F8C471'

    color_onset = '#196F3D'
    color_wakeup = '#943126'

    fig, ax = plt.subplots(figsize=(20, 6))

    ax.plot(series.step, series[column], color=color)

    # Apply label
    for r in s_lbls[['event', 'step']].itertuples():
        ax.axvline(x=r.step,
                   ymin=min(0, ymin),
                   ymax=max(1, ymax),
                   linewidth=2.,
                   linestyle='--',
                   color=color_onset if r.event == 'onset' else color_wakeup)

    ax.set_xlabel('Step')
    ax.set_ylabel(column, color=color, fontsize = 14)
    fig.suptitle(f'{column} for {series.series_id.iloc[0]}', fontsize = 20)
    fig.autofmt_xdate()
    return fig


In [ ]:
series_id = trn_events.series_id[0]

s = trn_series.query(f"`series_id`=='{series_id}'")
s_lbls = trn_events.query(f"`series_id`=='{series_id}'")

In [ ]:
#test_series['anglez_norm'] = tst_series['anglez'] / 90.
trn_series['anglez_norm'] = trn_series['anglez'] / 90.

In [ ]:
filter_percentages = [0, 50, 75, 99.7, 99.9, 99.95, 99.99, 99.999]
s['anglez_norm'] = s['anglez'] / 90.
for i in  range(len(filter_percentages) ) : 
    s[f'enmo_clean{i}'] = fft_signal_clean(s.enmo.values, filter_percent=filter_percentages[i], show_plot=False, y_min=0, y_max=2, L_min_idx=0, L_max_idx=50);
    s[f'anglez_clean{i}'] = fft_signal_clean(s.anglez_norm.values, filter_percent=filter_percentages[i], show_plot=False, y_min=0, y_max=2, L_min_idx=0, L_max_idx=50);


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier

X, y = make_blobs(centers=[[0, 0], [1, 1]], random_state=61526, n_samples=1000)


def plot_forest(max_depth=1):
    plt.figure()
    ax = plt.gca()
    h = 0.02

    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    if max_depth != 0:
        forest = RandomForestClassifier(n_estimators=20, max_depth=max_depth,
                                        random_state=1).fit(X, y)
        Z = forest.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
        Z = Z.reshape(xx.shape)
        ax.contourf(xx, yy, Z, alpha=.4)
        ax.set_title("max_depth = %d" % max_depth)
    else:
        ax.set_title("data set")
    ax.scatter(X[:, 0], X[:, 1], c=np.array(['b', 'r'])[y], s=60)
    ax.set_xlabel("PCA 1")
    ax.set_ylabel("PCA 2")
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xticks(())
    ax.set_yticks(())


def plot_forest_interactive():
    from ipywidgets import interactive, IntSlider
    slider = IntSlider(min=0, max=5, step=1, value=0)
    return interactive(plot_forest, max_depth=slider)

In [ ]:
X.shape

In [ ]:
plot_forest_interactive()

In [ ]:
import pytz
def make_features(df):
    # parse the timestamp and create an "hour" feature
    print((pd.to_datetime(df['timestamp'])[0]))
    df['timestamp'] = pd.to_datetime(df['timestamp']).apply(lambda t : t.replace(tzinfo=None))
    df["hour"] = df["timestamp"].dt.hour
    
    periods = 20
    df["anglez"] = abs(df["anglez"])
    df["anglez_diff"] = df.groupby('series_id')['anglez'].diff(periods=periods).fillna(method="bfill").astype('float16')
    df["enmo_diff"] = df.groupby('series_id')['enmo'].diff(periods=periods).fillna(method="bfill").astype('float16')
    df["anglez_rolling_mean"] = df["anglez"].rolling(periods,center=True).mean().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_rolling_mean"] = df["enmo"].rolling(periods,center=True).mean().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["anglez_rolling_max"] = df["anglez"].rolling(periods,center=True).max().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_rolling_max"] = df["enmo"].rolling(periods,center=True).max().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["anglez_rolling_std"] = df["anglez"].rolling(periods,center=True).std().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_rolling_std"] = df["enmo"].rolling(periods,center=True).std().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["anglez_diff_rolling_mean"] = df["anglez_diff"].rolling(periods,center=True).mean().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_diff_rolling_mean"] = df["enmo_diff"].rolling(periods,center=True).mean().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["anglez_diff_rolling_max"] = df["anglez_diff"].rolling(periods,center=True).max().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_diff_rolling_max"] = df["enmo_diff"].rolling(periods,center=True).max().fillna(method="bfill").fillna(method="ffill").astype('float16')
    
    return df

features = ["hour",
            "anglez",
            "anglez_rolling_mean",
            "anglez_rolling_max",
            "anglez_rolling_std",
            "anglez_diff",
            "anglez_diff_rolling_mean",
            "anglez_diff_rolling_max",
            "enmo",
            "enmo_rolling_mean",
            "enmo_rolling_max",
            "enmo_rolling_std",
            "enmo_diff",
            "enmo_diff_rolling_mean",
            "enmo_diff_rolling_max",
           ]

In [ ]:
zzz_series = pd.read_parquet("/Users/thomaswynn/Desktop/sleep_kaggle/Zzzs_train_multi.parquet").iloc[::10, : ]

In [ ]:
import pytz

train   = make_features(zzz_series)

X_train = train[features]
y_train = train["awake"]


In [ ]:
X_train.to_csv("X_train_RF.csv")

In [ ]:
%%time

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=5,
                                    min_samples_leaf=300,
                                    random_state=42,n_jobs=-1, max_depth= 1)

classifier.fit(X_train, y_train)

# save some memory
gc.collect();

In [ ]:
from sklearn import tree
fn=list(X_train.columns)
cn= list(y_train.unique().astype(str))
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=800)
tree.plot_tree(classifier.estimators_[0],
               feature_names = fn, 
               class_names=cn,
               filled = True);
fig.patch.set_alpha(0)
fig.savefig('rf_individualtree.png', transparent= True)

In [ ]:
cn

In [ ]:
y_train.unique()

In [ ]:
fn=list(X_train.columns)
cn= list(y_train.unique().astype(str))
fig, axes = plt.subplots(nrows = 1,ncols = 3,figsize = (40,10), dpi = 450)
for index in range(0, 3):
    tree.plot_tree(classifier.estimators_[index],
                   feature_names = fn, 
                   class_names=cn,
                   filled = True,
                   ax = axes[index]);
    for line in axes[index].get_lines():
        line.set_color("white")
    axes[index].patch.set_alpha(0)
    axes[index].title.set_color("white")
    axes[index].set_title('Estimator: ' + str(index + 1), fontsize = 30)
fig.savefig('rf_5trees.png', transparent=True)